In [14]:
import boto3
import asyncio
import aiohttp
from aiogram import Bot, Dispatcher, types, F
from aiogram.fsm.storage.memory import MemoryStorage
import logging
import os
from aiogram import Router
import uuid


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
API_TOKEN = os.getenv("TG_TOKEN")
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher()

router = Router()

bucket_name = 'audio-bot-bucket'

SERVICE_ACCOUNT_ID = ""
SERVICE_ACCOUNT_PRIVATE_KEY = ""
MAX_FILE_SIZE = 25 * 1024 * 1024  # 5 MB in bytes

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id=SERVICE_ACCOUNT_ID,
    aws_secret_access_key=SERVICE_ACCOUNT_PRIVATE_KEY
)

@dp.message()
async def handle_audio(message: types.Message):
    try:
        if message.voice:
            audio_url = f"https://api.telegram.org/file/bot{API_TOKEN}/{await bot.get_file(message.voice.file_id).file_path}"
            logger.info(f"📁 Voice URL: {audio_url}")
        elif message.audio:
            audio_url = message.audio.file_id
            file_info = await bot.get_file(message.audio.file_id)
            audio_url = f"https://api.telegram.org/file/bot{API_TOKEN}/{file_info.file_path}"
            logger.info(f"📁 Audio URL: {audio_url}")
        async with aiohttp.ClientSession() as session:
            async with session.head(audio_url) as resp:
                if resp.status != 200:
                    await message.reply("⚠️ Unable to retrieve file information.")
                    return
                file_size = int(resp.headers.get('Content-Length', 0))

        if file_size > MAX_FILE_SIZE:
            await message.reply(f"⚠️ The file is too large! Maximum allowed size is {MAX_FILE_SIZE / (1024 * 1024)} MB.")
            return


        async with aiohttp.ClientSession() as session:
            async with session.get(audio_url) as resp:
                audio_data = await resp.read()
        user_id = message.from_user.id
        original_file_name = file_info.file_path.split('/')[-1]
        unique_id = str(uuid.uuid4())
        file_name = f"{user_id}_{original_file_name}_{unique_id}"

        s3.put_object(Bucket='audio-bot-bucket', Key=file_name, Body=audio_data, StorageClass='COLD')
    except Exception as e:
        logger.error(f"❌ Error processing audio: {e}")
        await message.reply("⚠️ Sorry, something went wrong while processing.")

async def on_startup(_):
    logger.info("Bot is starting...")

async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    try:
        logger.info("Starting bot...")
        asyncio.run(main())
    except KeyboardInterrupt:
        logger.error("Bot stopped by user")
    except Exception as e:
        logger.error(f"Bot crashed with error: {e}")

Failed to fetch updates - TelegramNetworkError: HTTP Client says - ServerDisconnectedError: Server disconnected
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 8146216531)
Failed to fetch updates - TelegramConflictError: Telegram server says - Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 8146216531)
Failed to fetch updates - TelegramConflictError: Telegram server says - Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
Sleep for 1.269747 seconds and try again... (tryings = 1, bot id = 8146216531)
Failed to fetch updates - TelegramConflictError: Telegram server says - Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
Sleep for 1.766380 seconds and try again... (tryings = 2, bot id = 8146216531)
Failed to fetch updates - TelegramConflictError: Telegram serv